# Classic Segmentation

<div class="custom-button-row">
    <a 
        class="custom-button custom-download-button" href="../../notebooks/4_python_basics/Intro_to_Python_II.ipynb" download>
        <i class="fas fa-download"></i> Download this Notebook
    </a>
    <a
    class="custom-button custom-download-button" href="https://colab.research.google.com/github/HMS-IAC/bobiac/blob/gh-pages/colab_notebooks/4_python_basics/Intro_to_Python_II.ipynb" target="_blank">
        <img class="button-icon" src="../../_static/logo/icon-google-colab.svg" alt="Open in Colab">
        Open in Colab
    </a>
</div>

In [1]:
# /// script
# requires-python = ">=3.10"
# ///

# Standard library imports (no need to declare in dependencies)

This notebook covers the following **core building blocks**:

| Chapter | Concept | Why it matters |
|---------|---------|----------------|
| 0 | Setup | Import necessary packages |
| 1 | Loading an Image | Open tif image and display it |
| 2 | Filtering | Learn how to process images to improve thresholding results |
| 3 | Thresholding | Learn how to use thresholding to generate a binary mask |
| 4 | Labeling | Learn how to label binary masks |
| 5 | Mask Curation | Learn how to use watershed segmentation to refine binary masks |
| 6 | Processing Many Images | Learn how to apply these processing steps to many images |



Each chapter has:

1. **Summary** – review core concepts from lecture.
2. **Exercise** – _your turn_ to write code!

***

## 0. Setup

**Concept.**  
We are going to be using existing Python packages for classic segmentation, so we need to specify them at the beginning of our code. This is called specifying our **dependencies**. For learning purposes, we will import everything here at the beginning, and then refer back to what each package is as it comes up. 

### Exercise: Run the following code block to specify our dependencies

In [ ]:
# specify dependencies
import tifffile
import ndv
from skimage.filters import gaussian
from skimage.filters import threshold_otsu
from skimage.color import label2rgb
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
import numpy as np
from skimage.segmentation import watershed

***

## 1. Loading an Image

**Concept.**
To work with an image in Python, we need to specify where the image file is so that we can **read**, or load, it. Once the image file is read, we can **view** it.  

### Specifying your file's path
Once you have found your file's path, you should assign it to a variable to make it easy to work with. Here's an example:
```python
file_path = '/Users/edelase/HMS Dropbox/Eva de la Serna/Eva_CITE_folder/projects/bobiac/lectures/classic_segmentation/img.tif'
```

**❗️CAUTION❗️** Be wary of spaces in folder names, as they sometimes cause terminal to add `\` or `/` to file directories where they should not be. It is best practice to always use `_` in folder names whenever you would have wanted to have a space.

**🗒️ NOTE:** Specifying file paths is a common task outside of image segmentation with Python, so some of you may have experience with this already. Note the terminology though. An individual file's location is a **path**. A folder's path containing an individual or multiple files is called a **directory**.

### Exercise: Run the code below to specify the image file's path

In [1]:
image_path = "/Users/edelase/HMS Dropbox/Eva de la Serna/Eva_CITE_folder/projects/bobiac/lectures/classic_segmentation/DAPI_wf_0.tif"

### Reading an image
There are many different ways to read image files in Python. In this lesson, we are using the Python package `tifffile`, to read .tif image files. In order to read an image with `tifffile`, we will need to import it, which we have already done in Setup, and the provide it with the image's path:
```python
import tifffile
tifffile.imread(image_path)
```
`tifffile.imread()` will use that `image_path` you inputted to find your file and read it. It will then return the read file. Since we will be wanting to work with this file, we should assign the output to a variable for easy reference: 
```python
raw_image = tifffile.imread(image_path)
```

### Exercise: Run the following code to read the image 

In [2]:
raw_image = tifffile.imread(image_path)

### Viewing the image
In Python, reading the image and viewing it are two separate actions. Now that we have read the image and assigned it to the variable `raw_image`, we can display, or view, it using `ndv`, which we imported in setup. We can do this as follows: 
```python
import ndv
ndv.imshow(raw_image)
```
`ndv.imshow()` will use that `raw_image` you inputted to display your image. It will then return the image to view. 

### Exercise: Run the following code to view the image 

In [ ]:
ndv.imshow(raw_image)

***

## 2. Filtering

**Concept.**  
**Filters** change image pixel values using a **specific mathematical operation** to smooth and reduce noise from images. They can help improve thresholding results. 

### Applying a filter to an image
There are many different filters we can apply. Here's a summary of the ones we covered in depth in lecture: 
| Filter Name | Description | How to import it | Documentation Link | 
|---------|---------|----------------| ----------------|
| mean filter | For a given kernel size, sums values in a list and and then divides by the total number of values | `from skimage.filters.rank import mean` | [skimage.filters.rank.mean](https://scikit-image.org/docs/0.25.x/api/skimage.filters.rank.html#skimage.filters.rank.mean) |
| Gaussian blur filter | For a given kernel size, multiply each value by a Gaussian profile weighting, then divide by the total number of values | `from skimage.filters import gaussian` | [skimmage.filters.gaussian](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.gaussian) |
| median filter | For a given kernel size, take the middle number in a sorted list of numbers | `from skimage.filters import median` | [skimage.filters.median](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.median) |


### Exercise: Write code to apply a Gaussian blur filter to `raw_image` and assign it to the variable `filtered_image`
Remember - we already imported the gaussian function from skimage.filters in Setup!

***

## 3. Thresholding

**Concept.**  
**Thresholding** is when we select a range of digital values, or **intensity values**, in the image. These selected values are how we define regions of the image we are interested in. 

### Defining a threshold
We need to define a minimum intensity cutoff which separates the **background** (what we don't care about) from the **foreground** (what we do care about). We can manually pick a an intensity value as this cutoff value like below:
```python
min_threshold = 50
```

However, manually changing the value assigned to `min_threshold` until we find an optimal intensity cutoff value is tedious and may vary between images in a dataset. Therefore, it is best practice to instead use established thresholding algorithms to automatically define an intensity cutoff value. `skimage.filters` contains many different types of thresholding algorithms, but from that we will be using the Otsu thresholding algorithm `threshold_otsu`. We can do that as follows: 
```python
from skimage.filters import threshold_otsu
min_threshold = threshold_otsu(filtered_image)
```
Here, `threshold_otsu()` will use that inputted `filtered_image` to calculate an intensity cutoff value. It will return the cutoff value assigned to the variable `min_threshold`. 

### Generating a binary mask
We now want to use this cutoff value to generate a **binary mask**, which is an image that has 0 values where there is background and 1 values where there is foreground. By generating the binary mask, we will be able to evaluate whether this `min_threshold` is a sufficient cutoff value. 

We can generate the binary mask by using the comparison operator `>`:
```python
binary_mask = filtered_image > min_threshold
```
Python will interpret this line of code by going pixel by pixel through `filtered_image` and assigning 1 values where a pixel is greater than `min_threshold` and assigning 0 values where a pixel is equal or less than `min_threshold`. The output will be the binary mask image, filled with 1s and 0s. Since this binary mask is something we will be working with, we should assign it a variable, such as `binary_mask`. 

### Exercise: Write code to threshold `filtered_image` and generate a binary image assigned to the variable `binary_image`
Remember - we already imported the threshold_otsu function from skimage.filters in Setup!

***

## 4. Mask Labeling

**Concept.**  
Now that we have a binary mask that has white, or value 1, pixels that match the image foreground and black, or value 0, pixels that match the image background, we need a way to distinguish individual objects within this mask. **Labeling** a mask is when we identify individual objects within a binary mask and assign them a unique identifier. 

### Labeling a binary mask
From `skimage.measure` we can use `label()` to label a curated binary mask. Here's how we do it, although remember that we already imported label from skimage.measure in Setup: 
```python
from skimage.measure import label
labeled_image = label(binary_mask)
```
Here, `label()` will take the inputted `binary_image` and count each connected object in the image and assign them a number starting from 1. It will then return an image where each object's pixels have the value of its object's assigned number. It will return the transformed image, so we should assign it a variable, like `labeled_image`!


### Exercise: Write code to label `binary_mask` and assign it to the variable `labeled_image`
Remember - we already imported the label function from skimage.measure in Setup!

### Displaying a labeled mask on top of the image
Let's now summarize our final segmentation result in 1 image by viewing the `labeled_image` overlaid onto the original `raw_image`. From `skimage.color`, we can use `label2rgb` to do this, although remember we already imported label2rgb from skimage.color in Setup: 
```python
from skimage.color import label2rgb
seg_summary = label2rgb(labeled_image, image = raw_image, bg_label=0)
```
Here, `label2rgb()` is filled with a few arguments. First, we input the labeled mask `labeled_image`. Then, we specify the original image we want the `labeled_image` overlaid onto by saying `image = raw_image`. Finally, we specify `bg_label=0` to make the background of `labeled_image` transparent when it's overlaid onto `raw_image`. The output will be an rgb image of the labeled mask overlaid onto the raw image. Assign this result a variable for easy reference, say seg_summary!

### Exercise: Write code that displays `labeled_image` overlaid onto `raw_image`
Remember - we already imported the label function from skimage.measure in Setup!

***

## 5. Mask Curation

**Concept.**  
**Mask curation** is when additional processing steps are applied to a binary mask to more accurately match the image foreground. In the context of our nuclei example image, we need to apply additional processing steps to separate touching nuclei.

### Common mask curation steps
There are many different ways we can curate a binary mask. The table below summarizes very common curation steps:
| Algorithm Name | Description | How to import it | Documentation Link |
|---------|---------|----------------|----------------|
| Watershed Transform | A useful algorithm for separating touching objects. The output is a labeled image. | `from skimage.segmentation import watershed` | [skimage.segmentation.watershed](https://scikit-image.org/docs/0.25.x/api/skimage.segmentation.html#skimage.segmentation.watershed) |
| Morphological Opening | Mathematical operation that results in small object removal | `from skimage.morphology import binary_opening` | [skimage.morphology.binary_opening](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.binary_opening) |
| Morphological Closing | Mathematical operation that results in small hole removal | `from skimage.morphology import binary_closing` | [skimage.morphology.binary_closing](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.binary_closing) |
| Morphological Erosion | Mathematical operation that reduces shape size | `from skimage.morphology import binary_erosion` | [skimage.morphology.binary_erosion](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.binary_erosion) |
| Morphological Dilation | Mathematical operation that increases shape size | `from skimage.morphology import binary_dilation` | [skimage.morphology.binary_dilation](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.binary_dilation) |

### Applying a Watershed Transformation on an Image
Let's apply the Watershed Transformation to our `binary_mask` to separate any touching nuclei. From `skimage.segmentation` we can use `watershed()` to do this, although remember that we already imported watershed from skimage.segmentation in Setup: 
```python
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
import numpy as np
from skimage.segmentation import watershed

distance_transform = ndi.distance_transform_edt(binary_mask)
peak_coords = peak_local_max(distance_transform, footprint=np.ones((3, 3)), labels=binary_mask)
distance_image = np.zeros(distance_transform.shape, dtype=bool)
distance_image[tuple(peak_coords.T)] = True
markers, _ = ndi.label(distance_image)
labeled_ws_image = watershed(-distance_transform, markers, mask=binary_mask)
```
Here, `watershed()` will apply the Watershed Transform to the inputted `binary_image`. It will return the transformed, labeled image, so we should assign it a variable, say labeled_ws_image!

### Exercise: Write code to apply a watershed transform to `binary_mask` and assign it to the variable `labeled_ws_image`. Then, display `labeled_ws_image` overlaid onto `raw_image`
Remember, all packages have already been imported in Setup!

***

## 6. Processing Many Images

**Concept.**  
Something about statistics and working smarter not harder.

### Using a for loop to apply processing steps to many images

```python
from pathlib import Path
folder_path = Path(“/Users/edelase/bobiac”) # update with your folder's path
for image_path in path.iterdir():
    # read each image and apply processing steps
```

### Exercise: Improve the code below! 
1. Group processing steps into functions
2. Add a for loop so processing steps can be applied to many images

In [ ]:
# specify dependencies
import tifffile
import ndv

# load the image
image_path = "/Users/edelase/HMS Dropbox/Eva de la Serna/Eva_CITE_folder/projects/bobiac/lectures/classic_segmentation/DAPI_wf_0.tif"
raw_image = tifffile.imread(image_path)

# filter the image with gaussian filter
filtered_image = gaussian(raw_image)

# threshold filtered_image to generate binary mask
binary_mask = filtered_image > threshold_otsu(filtered_image)

# apply watershed to separate nuclei and label mask
distance_transform = ndi.distance_transform_edt(binary_mask)
peak_coords = peak_local_max(
    distance_transform, footprint=np.ones((3, 3)), labels=binary_mask
)
distance_image = np.zeros(distance_transform.shape, dtype=bool)
distance_image[tuple(peak_coords.T)] = True
markers, _ = ndi.label(distance_image)
labeled_ws_image = watershed(-distance_transform, markers, mask=binary_mask)

# display resulting labeled_image overlaid onto raw_image
seg_summary = label2rgb(labeled_ws_image, image=raw_image, bg_label=0)